# LangChain 기초 실습

## 1. langchain 패키지 설치
- openai api는 파이썬용 SDK 를 설치하거나 HTTP Client 를 사용하여 REST API 를 직접 호출하는 방식으로 사용할 수 있습니다.
- 프로그래밍 언어 선택에 제약이 있는 경우를 제외하면 대부분 파이썬 SDK 를 설치하여 사용합니다.

In [1]:
%%capture
!pip install langchain openai

## 2. Azure OpenAI 리소스 정보 설정
- openai API 를 호출하기 위해서는 API key 를 입력해야합니다.
- Azure OpenAI 의 경우, API key 뿐만 아니라 endpoint, deployment name 등이 추가로 필요합니다.
- 이 실습 환경에서는 Azure OpenAI 를 사용하기 위해 필요한 정보를 환경변수로 제공합니다.

In [2]:
#실습용 AOAI 환경변수 읽기
import os

AOAI_ENDPOINT=os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY=os.getenv("AOAI_API_KEY")
AOAI_DEPLOY_GPT4O=os.getenv("AOAI_DEPLOY_GPT4O")
AOAI_DEPLOY_GPT4O_MINI=os.getenv("AOAI_DEPLOY_GPT4O_MINI")
AOAI_DEPLOY_EMBED_3_LARGE=os.getenv("AOAI_DEPLOY_EMBED_3_LARGE")
AOAI_DEPLOY_EMBED_3_SMALL=os.getenv("AOAI_DEPLOY_EMBED_3_SMALL")
AOAI_DEPLOY_EMBED_ADA=os.getenv("AOAI_DEPLOY_EMBED_ADA")

## 3. 간단한 Chat Completion 구현
- AzureOpenAI 클라이언트를 생성하고 chat completions create 를 호출하여 간단히 GPT 의 응답을 받을 수 있습니다.

In [3]:
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=AOAI_ENDPOINT,
    azure_deployment=AOAI_DEPLOY_GPT4O_MINI,
    api_version="2024-10-21",
    api_key=AOAI_API_KEY
)

messages = [
    ("system", "You are a helpful assistant that translates English to Korean. Translate the user sentence.",),
    ("human", "I love programming."),
]
ai_msg = model.invoke(messages)
ai_msg

AIMessage(content='저는 프로그래밍을 사랑합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 31, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ded0d14823', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_materia

- LangChain 은 LLM 의 답변을 AIMessage라는 객체 형태로 출력합니다.
- content에는 실제 답변이 저장되며, response_metadata를 통해 토큰 사용량이나 필터링 정보 등을 추가로 얻을 수 있습니다.

In [4]:
print(ai_msg.content)

저는 프로그래밍을 사랑합니다.


In [5]:
from pprint import pprint
pprint(ai_msg.response_metadata)

{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
                            'protected_material_code': {'detected': False,
                                                        'filtered': False},
                            'protected_material_text': {'detected': False,
                                                        'filtered': False},
                            'self_harm': {'filtered': False,
                                          'severity': 'safe'},
                            'sexual': {'filtered': False, 'severity': 'safe'},
                            'violence': {'filtered': False,
                                         'severity': 'safe'}},
 'finish_reason': 'stop',
 'logprobs': None,
 'model_name': 'gpt-4o-mini-2024-07-18',
 'prompt_filter_results': [{'content_filter_results': {'hate': {'filtered': False,
                                                                'severity': 'safe'},
                                         

## 4. Multi-Modal (GPT-Vision) 구현
- o1, GPT-4o, GPT-4o-mini 및 GPT-4 Turbo with Vision 모델은 이미지를 분석하여 자연어로 답변할 수 있습니다.

In [6]:
#이미지 확인
from IPython.display import display, Image

image_url = "https://img.animalplanet.co.kr/news/2025/01/09/700/tx7yr5v253qpymae44c4.jpg"
display(Image(url=image_url, width=400))

In [7]:
message = HumanMessage(
    content=[
        {"type": "text", "text": "이미지를 설명해줘"},
        {"type": "image_url", "image_url": {"url": image_url}},
    ],
)
response = model.invoke([message])
print(response.content)

이 이미지는 귀엽고 사랑스러운 고양이가 편안하게 누워서 잠을 자고 있는 모습입니다. 고양이는 두 가지 색깔의 털을 가지고 있는데, 머리와 부분적인 몸이 주황색이고 나머지 부분은 흰색입니다. 특히 고양이는 몹시 편안해 보이며, 눈을 감고 미소 짓는 듯한 모습이 매력적입니다. 목에는 노란색 꽃 장식의 목걸이를 하고 있어 더욱 귀여운 느낌을 줍니다. 배경은 회색의 패브릭 소파로, 고양이의 털색과 잘 어울리며 아늑한 분위기를 연출하고 있습니다. 전체적으로 이 이미지는 평화롭고 따뜻한 느낌을 줍니다.


## 5. Message 객체의 종류 

- **SystemMessage**: AI 모델의 동작을 조정하는 역할로, 대화의 컨텍스트나 톤을 설정하는 메시지. 일부 모델에서는 시스템 메시지를 별도 API 파라미터로 전달.

- **HumanMessage**: 사용자가 입력한 내용을 나타내는 메시지로, 주로 텍스트 기반이지만 일부 모델은 이미지, 오디오 등의 멀티모달 데이터도 지원.

- **AIMessage**: 모델의 응답을 나타내며, 텍스트뿐만 아니라 도구 호출 요청 등의 데이터를 포함할 수 있음. 응답 메타데이터와 토큰 사용량 정보 등을 포함하기도 함.

- **AIMessageChunk**: AIMessage의 스트리밍 버전으로, 실시간으로 응답을 받을 수 있도록 청크 단위로 전송되며, 여러 청크를 합쳐 최종 메시지를 생성 가능.

- **ToolMessage**: 도구 호출 결과를 포함하는 메시지로, 호출된 도구의 ID 및 실행 결과와 관련된 추가 데이터(artifact 등)를 저장.

각 메시지는 역할(role), 콘텐츠(content), 메타데이터(metadata) 등의 정보를 포함하며, LangChain은 이를 다양한 챗 모델에서 공통적으로 사용할 수 있도록 표준화하였습니다.

(https://python.langchain.com/docs/concepts/messages/)

## 6. LCEL

LCEL(LangChain Expression Language)은 LangChain에서 체인(Chain)과 프롬프트(Prompt)를 보다 직관적이고 유연하게 조합할 수 있도록 지원하는 언어입니다.

LLM 에 프롬프트를 입력하고 출력 데이터를 정제하는 일련의 과정을 하나의 파이프라인으로 묶을 수 있습니다.

1. **PromptTemplate**  
   - 입력을 동적으로 받아 특정 형식의 프롬프트를 생성합니다.

2. **LLM**  
   - LLM 객체가 될 수도 있고 또는 chain 끼리 연결할 수도 있습니다.

3. **Output Parser**
   - AIMessage 를 원하는 포맷으로 변환하여 받을 수 있습니다.
   - StrOutputParser
   - JsonOutputParser


In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{text} 를 영어로 번역해주세요.")
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text} 를 영어로 번역해주세요.')

In [9]:
prompt.format(text="한국의 수도는 서울입니다.")

'한국의 수도는 서울입니다. 를 영어로 번역해주세요.'

In [10]:
chain1 = prompt | model
chain1.invoke({"text": "한국의 수도는 서울입니다"})

AIMessage(content='"The capital of South Korea is Seoul."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 20, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': Fals

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [13]:

chain2 = prompt | model | output_parser
chain2.invoke({"text": "한국의 수도는 서울입니다"})

'"The capital of South Korea is Seoul."'

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class Translate(BaseModel):
    origin: str = Field(description="번역 전 텍스트")
    translated: str = Field(description="번역된 텍스트")

json_parser = JsonOutputParser(pydantic_object=Translate)

json_prompt = PromptTemplate(
    template="사용자의 text를 영어로 번역하세요.\n{format_instructions}\n{text}\n",
    input_variable=["text"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

chain3 = json_prompt | model | json_parser
chain3.invoke({"text": "한국의 수도는 서울입니다"})

{'origin': '한국의 수도는 서울입니다',
 'translated': 'The capital of South Korea is Seoul.'}

## [부록] Structured Outputs

JsonOutputParser 는 프롬프트를 기반으로 LLM 이 Json 으로 답변한 것을 파싱하여 Json 값으로 반환해주지만 LLM 이 실수할 수 있습니다.

Structured Output 은 LLM 이 JSON 스키마를 준수하여 답변하도록 OpenAI API가 보장해줍니다.

- 신뢰할 수 있는 유형 안전성: 잘못 포맷된 응답을 검증하거나 다시 시도할 필요가 없습니다.
- 명시적 거부: 안전 기반 모델 거부는 이제 프로그래밍 방식으로 감지할 수 있습니다.
- 더 간단한 프롬프트: 일관된 형식을 달성하기 위해 강력한 표현의 프롬프트가 필요하지 않습니다.

https://python.langchain.com/docs/concepts/structured_outputs/

https://platform.openai.com/docs/guides/structured-outputs

In [15]:
class Translate(BaseModel):
    origin: str = Field(description="번역 전 텍스트")
    translated: str = Field(description="번역된 텍스트")

model_with_structure = model.with_structured_output(Translate)
model_with_structure.invoke("한국의 수도는 서울입니다. 를 영어로 번역해줘")

Translate(origin='한국의 수도는 서울입니다.', translated='The capital of Korea is Seoul.')

## 7. ChatPromptTemplate
- 대화 히스토리를 프롬프트로 입력할 때 활용합니다.
- "system" : 전역에 반영되는 시스템 프롬프트
- "human" : 사용자 입력 메시지
- "ai" : LLM 의 답변 메시지

In [16]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 번역 업무를 수행하는 에이전트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워!"),
        ("ai", "안녕하세요, 반갑습니다."),
        ("human", "{user_input}")
    ]
)

chat_prompt.format_messages(
    user_input="너의 이름은 뭐야?",
    name="선경"
)

[SystemMessage(content='당신은 번역 업무를 수행하는 에이전트입니다. 당신의 이름은 선경 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 반갑습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='너의 이름은 뭐야?', additional_kwargs={}, response_metadata={})]

In [17]:
chain4 = chat_prompt | model | StrOutputParser()
chain4.invoke({"user_input": "너의 이름은 뭐야?", "name": "선경"})

'제 이름은 선경입니다. 당신과 대화하게 되어 기쁩니다!'

## 8. MessagePlaceholder
- 채팅 기록이나 특정 메시지 데이터를 동적으로 프롬프트에 넣을 때 사용합니다.

In [32]:
from langchain_core.prompts import MessagesPlaceholder

chat_prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),  # 이전 대화 기록을 저장하는 플레이스홀더
    ("human", "{input}")  # 사용자 입력
])

chain5 = chat_prompt2 | model | StrOutputParser()

messages = [
    {"role": "human", "content": "오늘 점심메뉴 추천해줘!"},
    {"role": "ai", "content": "해장국 어떠신가요?"}
]

chain5.invoke({"chat_history": messages, "input": "다른 메뉴는 없을까?"})

'물론이죠! 다양한 메뉴로 추천해드릴게요:\n\n1. **비빔밥** - 신선한 채소와 고기를 밥과 함께 비벼 먹는 맛있는 한 그릇 요리입니다.\n2. **김치찌개** - 따끈하고 얼큰한 김치찌개에 밥과 함께 즐기면 좋습니다.\n3. **볶음밥** - 냉장고에 남은 재료를 활용해 볶음밥을 만들어보세요! 간단하지만 맛있습니다.\n4. **족발** - 쫄깃한 족발을 추천해드려요! 쌈채소와 함께 먹으면 더욱 맛있습니다.\n5. **샐러드와 파스타** - 가벼운 식사를 원하시면 신선한 샐러드와 함께 크림 또는 토마토 소스 파스타를 조합해보세요.\n\n오늘은 어떤 메뉴가 마음에 드세요?'